### Install all required packages:

In [ ]:
! pip install micom 
! pip install numpy Cython
! pip install biom-format

### Import all required libraries and functions:

In [3]:
#for data manipulation:
import pandas as pd
import numpy as np
import pickle
#for building community:
from micom.workflows import build
from micom import Community
from micom.qiime_formats import load_qiime_medium
from micom.workflows import grow
#for visualizing results:
from micom.viz import plot_growth
from micom.viz import plot_exchanges_per_sample
from micom.viz import plot_exchanges_per_taxon
from micom.viz import plot_association
from micom.measures import production_rates

### Import required data:


In [ ]:
data = pd.read_csv('data_long_form.csv') #genera data in long form format
df_meta = pd.read_csv('fran_metadata.csv') #to obtain disease group of each sample

#add disease group column onto the data dataframe:
disease = list(df_meta['Disease.Group'])
data['disease_stat'] = disease*74

#### The resulting data frame should have:

Column 1) sample_id - the sample each measure corresponds to

Column 2) genus - the genus being measured

Column 3) relative - the relative abunance of the genus for that sample

Column 4) id- same as genus essentially but required by the model

Column 5) disease_status - if the sample the measure comes from has IBD or not. 

Optional: Column 6) The family corresponding to each genus.

### The long form data frame was made via:

In [ ]:
data_genus = 'fran_genera_diff.csv' #data on the 74 selected genus only
df_genus = pd.read_csv(format(data_genus))
#Rename sample column and index it:
df_genus.rename(columns={'Unnamed: 0': 'Sample'}, inplace=True)
df_genus= df_genus.set_index('Sample')

#melt:
dfm = df_genus.melt(ignore_index=False).reset_index()
dfm.rename(columns={'Sample': 'sample_id', 'value': 'relative', 'variable': 'genus'}, inplace=True)
dfm['id'] = dfm['genus']
dfm.to_csv('data_long_form.csv')

#### Obtain required AGORA database and diet data:

In [ ]:
!wget -O agora103_genus.qza https://zenodo.org/record/3755182/files/agora103_genus.qza?download=1
!wget -O western_diet_gut.qza https://zenodo.org/record/3755182/files/western_diet_gut.qza?download=1

## Builiding the manifest

In [ ]:
manifest = build(data, "agora103_genus.qza", "gen_mod", solver="osqp",
                 cutoff=0, threads=2) #gen_mod is the name of a folder to store the results to 

## Obtaining the growth model:

In [ ]:
#load the diet data:
medium = load_qiime_medium("western_diet_gut.qza")

growth_results = grow(manifest, "gen_mod", medium, tradeoff=0.5, threads=2) #store to the same folder

# Save the individual results to a file:
pickle.dump(growth_results, open("growth.pickle", "wb"))


## Visualising results:

In [ ]:
#Visualize the growth rates:
pl = plot_growth(growth_results, filename="growth_rates.html")

#Visulaize how the metabolites are exchanged between samples:
pl = plot_exchanges_per_sample(growth_results, filename="consumption.html")

#Visualize how the metabolites are exchaned between the genus:
pl = plot_exchanges_per_taxon(growth_results, filename="niche.html")


## Testing for significant metabolites:

In [ ]:
#extract production rates:
prod = production_rates(growth_results)

#set phenotype to the disease group:
phenotype = manif.disease_stat

#find associations between metabolites and disease:
pl = plot_association(growth_results,phenotype= phenotype,variable_type="binary",filename="association.html",fdr_threshold=0.5)